<a href="https://colab.research.google.com/github/hafsaaaaaaaa/MLPROJECT_/blob/main/project_pnenumonia_or_normal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!mkdir ~/.kaggle

In [3]:
!mv kaggle.json ~/.kaggle
!chmod 600 ~/.kaggle/kaggle.json

In [4]:
!kaggle datasets download -d paultimothymooney/chest-xray-pneumonia

 99% 2.27G/2.29G [00:11<00:00, 257MB/s]
100% 2.29G/2.29G [00:11<00:00, 218MB/s]


In [ ]:
!unzip "/content/chest-xray-pneumonia.zip"

In [17]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define paths
train_dir = "/content/chest_xray/train"
validation_dir = "/content/chest_xray/val"
test_dir = "/content/chest_xray/test"

# Data augmentation and normalization for training
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

# Normalization for validation and testing
validation_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Define batch size
batch_size = 32

# Flow training images in batches using train_datagen generator
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=batch_size,
    class_mode='binary')

# Flow validation images in batches using validation_datagen generator
validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(150, 150),
    batch_size=batch_size,
    class_mode='binary')

# Flow test images in batches using test_datagen generator
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(150, 150),
    batch_size=batch_size,
    class_mode='binary')

# Display class labels
print("Class labels:", train_generator.class_indices)

# Example of how to use these generators with a CNN model in TensorFlow/Keras
# Define and compile your CNN model
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
# Calculate steps per epoch
steps_per_epoch_train = train_generator.samples // batch_size
steps_per_epoch_validation = validation_generator.samples // batch_size

# Ensure at least one step per epoch
if steps_per_epoch_train == 0:
    steps_per_epoch_train = 1
if steps_per_epoch_validation == 0:
    steps_per_epoch_validation = 1

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch_train,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=steps_per_epoch_validation)





# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(test_generator, steps=test_generator.samples // batch_size)
print("Test Accuracy:", test_acc)


Found 5216 images belonging to 2 classes.
Found 16 images belonging to 2 classes.
Found 624 images belonging to 2 classes.
Class labels: {'NORMAL': 0, 'PNEUMONIA': 1}
Epoch 1/10
163/163 [==============================] - 72s 426ms/step - loss: 0.4731 - accuracy: 0.7830 - val_loss: 0.9542 - val_accuracy: 0.6250
Epoch 2/10
163/163 [==============================] - 69s 422ms/step - loss: 0.3197 - accuracy: 0.8570 - val_loss: 0.4982 - val_accuracy: 0.8125
Epoch 3/10
163/163 [==============================] - 69s 422ms/step - loss: 0.2806 - accuracy: 0.8808 - val_loss: 0.7871 - val_accuracy: 0.7500
Epoch 4/10
163/163 [==============================] - 68s 415ms/step - loss: 0.2439 - accuracy: 0.8924 - val_loss: 0.6111 - val_accuracy: 0.8125
Epoch 5/10
163/163 [==============================] - 69s 420ms/step - loss: 0.2269 - accuracy: 0.9074 - val_loss: 0.6463 - val_accuracy: 0.8125
Epoch 6/10
163/163 [==============================] - 68s 418ms/step - loss: 0.2166 - accuracy: 0.9105 - val

In [16]:
# Check if the training directory contains images
num_train_samples = sum(len(files) for _, _, files in os.walk(train_dir))
print("Number of training samples:", num_train_samples)

# Check the value of train_generator.samples
print("Number of training generator samples:", train_generator.samples)


Number of training samples: 5216
Number of training generator samples: 5216


In [18]:
# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(test_generator, steps=test_generator.samples // batch_size)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_acc)


19/19 [==============================] - 4s 211ms/step - loss: 0.2636 - accuracy: 0.9243
Test Loss: 0.2636030316352844
Test Accuracy: 0.9243420958518982


In [19]:
from tensorflow.keras.preprocessing import image
import numpy as np

# Path to the image you want to test
image_path = "/content/chest.jpg"  # Replace this with the path to your image

# Load and preprocess the image
img = image.load_img(image_path, target_size=(150, 150))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array = img_array / 255.0  # Normalize the image

# Predict the class probabilities
predictions = model.predict(img_array)

# Print the predicted class probabilities
print("Predictions:", predictions)


1/1 [==============================] - 0s 316ms/step
Predictions: [[0.91562295]]


In [20]:
# Predict the class probabilities
predictions = model.predict(img_array)

# Interpret the predictions
if predictions[0] < 0.5:
    print("Predicted: Normal")
else:
    print("Predicted: Pneumonia")


1/1 [==============================] - 0s 18ms/step
Predicted: Pneumonia


In [21]:
model.save("model.h5")


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [22]:
from tensorflow.keras.preprocessing import image
import numpy as np

# Path to the image you want to test
image_path = "/content/normal chest.jpg"  # Replace this with the path to your image

# Load and preprocess the image
img = image.load_img(image_path, target_size=(150, 150))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array = img_array / 255.0  # Normalize the image

# Predict the class probabilities
predictions = model.predict(img_array)

# Print the predicted class probabilities
print("Predictions:", predictions)


1/1 [==============================] - 0s 18ms/step
Predictions: [[0.06389415]]


In [23]:
# Predict the class probabilities
predictions = model.predict(img_array)

# Interpret the predictions
if predictions[0] < 0.5:
    print("Predicted: Normal")
else:
    print("Predicted: Pneumonia")


1/1 [==============================] - 0s 18ms/step
Predicted: Normal
